# Import the libraries

In [9]:
# Warning control
import warnings

warnings.filterwarnings("ignore")

import os

from dotenv import load_dotenv

_ = load_dotenv() # read local .env file

from huggingface_hub import login

login(os.environ['HF_API_KEY'])

# First we make a few tools like sum and multiply

In [10]:
# First we make a few tools
from smolagents import tool

@tool
def sum_two_numbers(x: int, y: int) -> int:
    """
    Sum two numbers

    Args:
        x: the first number
        y: the second number
    """
    return x + y

@tool
def multiply_two_numbers(x: int, y: int) -> int:
    """
    Multiply two numbers

    Args:
        x: the first number
        y: the second number
    """
    
    return x * y

# Create CodeAgent by using HuggingFace Code Agent

In [25]:
from smolagents import HfApiModel, CodeAgent

model = HfApiModel(
    "meta-llama/Llama-4-Scout-17B-16E-Instruct",
    provider="sambanova", # Choose a specific inference provider
    max_tokens=4096,
    temperature=0.1
)

In [26]:
agent = CodeAgent(
    model=model,
    tools=[sum_two_numbers, multiply_two_numbers],
    max_steps=10,
    verbosity_level=2
)
agent.logger.console.width=66

# Run the agent for a single task which uses the tools

In [27]:
agent.run(
    """Can you get me the sum of 2 and 3?"""
)

╭─────────────────────────── New run ────────────────────────────╮
│                                                                │
│ Can you get me the sum of 2 and 3?                             │
│                                                                │
╰─ HfApiModel - meta-llama/Llama-4-Scout-17B-16E-Instruct ───────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ───────────────────────────────────────
Thought: I will use the `sum_two_numbers` tool to calculate the   
sum of 2 and 3.                                                   
                                                                  
Code:                                                             
```py                                                             
result = sum_two_numbers(x=2, y=3)                                
final_answer(result)                                              
```<end_code>                                                     

─ Executing parsed code: ─────────────────────────────────────── 
  result = sum_two_numbers(x=2, y=3)                              
  final_answer(result)                                            
 ────────────────────────────────────────────────────────────────

Out - Final answer: 5

[Step 1: Duration 2.11 seconds| Input tokens: 2,069 | Output 
tokens: 46]

5

# Run the agent for a complex task which uses the tool along with more steps

In [64]:
agent.run(
    """Can you get me the maximum of sum of these pairs of numbers [(2, 3), (4, 5), (6, 7), (8, 9), (10, 11)]?"""
)

╭─────────────────────────── New run ────────────────────────────╮
│                                                                │
│ Can you get me the maximum of sum of these pairs of numbers    │
│ [(2, 3), (4, 5), (6, 7), (8, 9), (10, 11)\]?                   │
│                                                                │
╰─ HfApiModel - meta-llama/Llama-4-Scout-17B-16E-Instruct ───────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ───────────────────────────────────────
Thought: To find the maximum sum of pairs of numbers, I will first
calculate the sum of each pair, then find the maximum sum among   
them.                                                             
                                                                  
Code:                                                             
```py                                                             
pairs = [(2, 3), (4, 5), (6, 7), (8, 9), (10, 11)]                
sums = [sum(pair) for pair in pairs]                              
max_sum = max(sums)                                               
print(max_sum)                                                    
```<end_code>                                                     

─ Executing parsed code: ─────────────────────────────────────── 
  pairs = [(2, 3), (4, 5), (6, 7), (8, 9), (10, 11)]              
  sums = [sum(pair) for pair in pairs]                            
  max_sum = max(sums)                                             
  print(max_sum)                                                  
 ────────────────────────────────────────────────────────────────

Execution logs:
21

Out: None

...

Output message of the LLM: ───────────────────────────────────────
Thought: The maximum sum of the pairs is 21, which is the sum of  
the pair (10, 11). I will now provide this as the final answer.   
                                                                  
Code:                                                             
```py                                                             
final_answer(21)                                                  
```<end_code>                                                     

─ Executing parsed code: ─────────────────────────────────────── 
  final_answer(21)                                                
 ────────────────────────────────────────────────────────────────

Out - Final answer: 21

[Step 2: Duration 1.24 seconds| Input tokens: 4,419 | Output 
tokens: 141]

21

# Create a normal tooling agent and try running the same prompt

In [62]:
from smolagents import ToolCallingAgent

model_tool_calling = HfApiModel(
    "Qwen/Qwen2.5-72B-Instruct",
    provider="together",
    temperature=0.6
)
agent_tool_calling = ToolCallingAgent(
    model=model_tool_calling,
    tools=[sum_two_numbers, multiply_two_numbers],
    max_steps=20,
)
agent_tool_calling.logger.console.width=66

In [59]:
agent_tool_calling.run(
    """Can you get me the sum of 2 and 3?"""
)

╭─────────────────────────── New run ────────────────────────────╮
│                                                                │
│ Can you get me the sum of 2 and 3?                             │
│                                                                │
╰─ HfApiModel - Qwen/Qwen2.5-72B-Instruct ───────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭────────────────────────────────────────────────────────────────╮
│ Calling tool: 'sum_two_numbers' with arguments: {'x': 2, 'y':  │
│ 3}                                                             │
╰────────────────────────────────────────────────────────────────╯

Observations: 5

[Step 1: Duration 1.39 seconds| Input tokens: 1,290 | Output 
tokens: 27]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': '5'}   │
╰────────────────────────────────────────────────────────────────╯

Final answer: 5

[Step 2: Duration 0.79 seconds| Input tokens: 2,674 | Output 
tokens: 47]

'5'

In [63]:
agent_tool_calling.run(
    """Can you get me the maximum of sum of these pairs of numbers [(2, 3), (4, 5), (6, 7), (8, 9), (10, 11)]?"""
)

╭─────────────────────────── New run ────────────────────────────╮
│                                                                │
│ Can you get me the maximum of sum of these pairs of numbers    │
│ [(2, 3), (4, 5), (6, 7), (8, 9), (10, 11)\]?                   │
│                                                                │
╰─ HfApiModel - Qwen/Qwen2.5-72B-Instruct ───────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭────────────────────────────────────────────────────────────────╮
│ Calling tool: 'sum_two_numbers' with arguments: {'x': 2, 'y':  │
│ 3}                                                             │
╰────────────────────────────────────────────────────────────────╯

Observations: 5

[Step 1: Duration 0.83 seconds| Input tokens: 1,323 | Output 
tokens: 27]

...

Error while generating or parsing output:
The JSON blob you used is invalid

[Step 20: Duration 0.97 seconds| Input tokens: 37,351 | Output 
tokens: 224]

Reached max steps.

[Step 21: Duration 2.36 seconds| Input tokens: 38,553 | Output 
tokens: 322]

'To find the maximum sum of the given pairs of numbers, we can simply calculate the sum of each pair and then determine the largest sum. Here are the sums:\n\n- 2 + 3 = 5\n- 4 + 5 = 9\n- 6 + 7 = 13\n- 8 + 9 = 17\n- 10 + 11 = 21\n\nThe maximum sum is **21**.'